In [1]:
import numpy as np
from scipy.special import factorial
import scipy.optimize as opt
from scipy.optimize import minimize
from scipy.stats import poisson
from scipy.optimize import differential_evolution
import pandas as pd
import os

In [2]:
def level_k_choice_probabilities(expected_utilities, gamma, k):
    lambda_k = gamma * k  
    util = np.max(lambda_k * expected_utilities)  
    exp_utilities = np.exp(lambda_k * expected_utilities - util)
    return exp_utilities / np.sum(exp_utilities)  

 
def calculate_expected_utilities(payoff_matrix, p_k_1): 
    return np.dot(payoff_matrix, p_k_1)


In [3]:
def poisson_weights(tau, level):
    levels = np.arange(0, level + 1)
    weights = poisson.pmf(levels, tau)
    return weights

def aggregate_choice_probabilities(payoff_matrix, tau, gamma, level):
    num_strategies = payoff_matrix.shape[0]  
    overall_probs = np.zeros(num_strategies)  

    weights = poisson_weights(tau, level)
    level_probs = np.ones(num_strategies) / num_strategies  
    overall_probs = weights[0] * level_probs
    
    for k in range(1, level + 1):
        expected_utilities = calculate_expected_utilities(payoff_matrix, level_probs)
        level_probs = level_k_choice_probabilities(expected_utilities, gamma, k)
        overall_probs += weights[k] * level_probs
        
    return overall_probs

In [4]:
def log_likelihood(params, payoff_matrices, observed_probs_list, level):
    tau, gamma = params
    log_likelihood_value = 0
    for payoff_matrix, observed_probs in zip(payoff_matrices, observed_probs_list):
        predicted_probs = aggregate_choice_probabilities(payoff_matrix, tau, gamma, level)
        predicted_probs = np.maximum(predicted_probs, 1e-8)
        log_likelihood_value += np.sum(observed_probs * np.log(predicted_probs))
               
    if np.isnan(log_likelihood_value):
        return 1e10 

    return -log_likelihood_value


In [5]:
# Define games
all_games = {
    "Competitive-Base": {
        "payoff_matrices": [
            np.array([[10, 0, -5], [-10, 5, 8], [0, 5, -5]]),
            np.array([[-10, 5, 8], [10, 0, -5], [0, -5, 5]])
        ],

    }
#and others
}
init = [(), (), ()]
results = []

In [ ]:
for game_name, game_data in all_games.items():
    payoff_matrices = game_data["payoff_matrices"]
    for model_name, observed_probs_list in game_data["observed_probs"].items():
        best_result = None
        best_log_likelihood = float("inf")
        for guess in init:
            level = 10
            result = minimize(
                log_likelihood,
                guess,
                args=(payoff_matrices, observed_probs_list, level),
                method='L-BFGS-B',
                bounds=[(None, level), (1e-8, None)],
                options={'disp': False}
            )

            if result.success and result.fun < best_log_likelihood:
                best_result = result
                best_log_likelihood = result.fun

        results.append({
            "Game": game_name,
            "Model": model_name,
            "Tau": best_result.x[0] if best_result else None,
            "Gamma": best_result.x[1] if best_result else None,
            "Log-Likelihood": -best_log_likelihood if best_result else None
        })

results_df = pd.DataFrame(results)
results_df.to_csv("results.csv", index=False)


In [ ]:
def estimate_parameters_for_game(game_name, payoff_matrices, csv_path, level=10):
    
    if not os.path.exists(csv_path):
        print(f"[WARNING] CSV for game '{game_name}' not found at '{csv_path}'. Skipping.")
        return None
    
    data = pd.read_csv(csv_path)
    data.rename(columns={data.columns[0]: "Demographic ID"}, inplace=True)

    data.iloc[:, 1:] = data.iloc[:, 1:] / 100.0
    m = payoff_matrices[0].shape[0]  
    n = payoff_matrices[1].shape[1]     
    results = []
    for index, row in data.iterrows():
        demographic_id = row.iloc[0]  
        observed_probs_player1 = row.iloc[1 : 1 + m].to_numpy()
        observed_probs_player2 = row.iloc[1 + m : 1 + m + n].to_numpy()       
        observed_probs_list = [observed_probs_player1, observed_probs_player2]
        
        best_result = None
        best_log_likelihood = float("inf")
        
        for guess in init:
            result = minimize(
                log_likelihood,
                guess,  
                args=(payoff_matrices, observed_probs_list, level),
                method='L-BFGS-B',
                bounds=[(None, level), (1e-8, None)],  
                options={'disp': False}
            )
            if result.success and result.fun < best_log_likelihood:
                best_result = result
                best_log_likelihood = result.fun

        if best_result is not None:
            results.append({
                "Model": demographic_id,
                "Tau": best_result.x[0],
                "Gamma": best_result.x[1],
                "Log-Likelihood": -best_log_likelihood
            })
        else:
            results.append({
                "Model": demographic_id,
                "Tau": None,
                "Gamma": None,
                "Log-Likelihood": None
            })
    
    results_df = pd.DataFrame(results)
    return results_df
def main_estimation_pipeline():
    all_results = []  
    
    for game_name, game_info in all_games.items():
        csv_file = f"sr/{game_name}_CoT.csv"
        
        payoff_matrices = game_info["payoff_matrices"]
        
        print(f"Running estimation for: {game_name}")
        results_df = estimate_parameters_for_game(
            game_name=game_name,
            payoff_matrices=payoff_matrices,
            csv_path=csv_file,
            level=10
        )
        
        if results_df is not None:
            out_csv = f"{game_name}_estimations.csv"
            results_df.to_csv(out_csv, index=False)
            print(f"Results saved -> {out_csv}")
            
            # Store results for display
            all_results.append((game_name, results_df))
            
        else:
            print(f"Skipping {game_name} (no CSV or no valid results).")
        
        print("-" * 50)
    print("\nSummary of Estimation Results:")
    for game_name, results_df in all_results:
        print(f"Game: {game_name}")
        print(results_df)
        print("-" * 50)

if __name__ == "__main__":
    main_estimation_pipeline()
